#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-11-10 15:51:42--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc778272cfa089eee6454fc26fa.dl.dropboxusercontent.com/cd/0/inline/CeI_vX0nBG6W18AacEkVBK3kGvkG4Me9EO0bbdeP6Qt8hSWKqcf6UMwV-4fNrN3_IgYwzpTS1FpfyjEeSyo9xxxHKXcnC8803OCPheEajthb0uZdDSl-3mu516dW_QEeJBE/file?dl=1# [following]
--2024-11-10 15:51:43--  https://ucc778272cfa089eee6454fc26fa.dl.dropboxusercontent.com/cd/0/inline/CeI_vX0nBG6W18AacEkVBK3kGvkG4Me9EO0bbdeP6Qt8hSWKqcf6UMwV-4fNrN3_IgYwzpTS1FpfyjEeSyo9xxxHKXcnC8803OCPheEajthb0uZdDSl-3mu516dW_QEeJBE/file?dl=1
Resolving ucc778272cfa089eee6454fc26fa.dl.dropboxusercontent.com (ucc778272cfa089eee6454fc26fa.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [7]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [9]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [10]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [11]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [12]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [13]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [14]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  likes = user_data['Conversation'][0]['conversation_1']['user_likes']
  likes_decode = []
  for movie in likes:
    likes_decode.append(item_map[movie])
  dislikes = user_data['Conversation'][0]['conversation_1']['user_dislikes']
  dislikes_decode = []
  for movie in dislikes:
    dislikes_decode.append(item_map[movie])
  rec_item = user_data['Conversation'][0]['conversation_1']['rec_item']
  rec_item_decode = item_map[rec_item[0]]
  prompt = generate_zeroshot_prompt(likes_decode,dislikes_decode)
  prompts.append(prompt)
  ground_truth.append(rec_item_decode)



In [15]:
answers = []
for prompt in prompts:
  ans = text_generator(prompt, max_new_tokens=30)
  answers.append(ans[0]['generated_text'])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [16]:
for answer in answers:
  print(answer)


<s>[INST] I've liked these movies: Spy: Susan Cooper Undercover
Identity Thief
 Knowing [Blu-ray]. I've disliked these movies: . Recommend a new movie for me to watch.[/INST] Based on your preference for comedies and action-thrillers, I would recommend "The Secret Service" (2014) star
<s>[INST] I've liked these movies: Bugville: Hoppity Goes to Town
The Little Shop of Horrors in Color
Missile to the Moon Colorized / Black &amp; White. I've disliked these movies: . Recommend a new movie for me to watch.[/INST] Based on your interest in animated films, musicals, and colorized classics, I would recommend "An American Tail" for its charming animation
<s>[INST] I've liked these movies: . I've disliked these movies: Ring of Darkness
Cabin Fever Region 2. Recommend a new movie for me to watch.[/INST] Based on your preferences, I would recommend "The Descent" (2005). It is a British horror film that shares some elements with
<s>[INST] I've liked these movies: A Nightmare on Elm Street 3 - Drea

## Lora

In [21]:
import os
import peft

#os.environ["CUDA_VISIBLE_DEVICES"] = "0" # si solo vamos a usar la primera GPU que tengamos

import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig, GPTJForCausalLM
from peft import LoraConfig, get_peft_model

In [23]:
# Settings for A100 - For 3090
MICRO_BATCH_SIZE = 4  # change to 8 for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 2e-5
CUTOFF_LEN = 256
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [24]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [25]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

## QLora

In [26]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [27]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [28]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3407872 || all params: 3761770496 || trainable%: 0.09059223585340173


In [33]:
import transformers

In [38]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

model = AutoModelForCausalLM.from_pretrained('model_name', load_in_4bit=True, device_map='auto')
lora_config = LoraConfig(r=8, lora_alpha=32, target_modules=['q_proj', 'v_proj'], lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# Entrenamiento
train_model(model, csr_dataset)


OSError: model_name is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`